# Extraction of policies, outcomes and correlations

Code for tagging and extraction of policies, outcomes and correlations.

## Import packages

In [1]:
# Enter your API key for OpenAI. 
import openai
openai.organization = ""
openai.api_key = ""

In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import numpy as np

## Import data

In [ ]:
# Import your data
input_path = ""

db_init = pd.read_csv(input_path)
db_init["abstract"] = db_init["abstract"].fillna("").astype(str)

In [ ]:
# Extract random sample for testing
random_rows = db_init.sample(n=1000)